In [1]:
import os

from projectaria_tools.core import data_provider, mps
from projectaria_tools.core.mps.utils import (
    filter_points_from_confidence,
    get_gaze_vector_reprojection,
    get_nearest_eye_gaze,
    get_nearest_pose,
)
from projectaria_tools.core.stream_id import StreamId
import numpy as np

folder = "eedc2456-ccee-477a-916e-70a3a15cd004_blur"

# Load the VRS file
vrsfile = os.path.join(folder, "eedc2456-ccee-477a-916e-70a3a15cd004_blur.vrs")

# Trajectory and global points
# closed_loop_trajectory = os.path.join(
#     folder, "slam", "closed_loop_trajectory.csv"
# )
# global_points = os.path.join("video1", "slam", "semidense_points.csv.gz")

# Eye gaze
generalized_eye_gaze_path = os.path.join(
    folder, "eyegaze", "general_eye_gaze.csv"
)
generalized_eye_gaze_depth_path = os.path.join(
    folder, "eyegaze", "gaze_with_depth.csv"
)

# Hand tracking
# wrist_and_palm_poses_path = os.path.join(
#     folder, "hand_tracking", "wrist_and_palm_poses.csv"
# )

# Create data provider and get T_device_rgb
provider = data_provider.create_vrs_data_provider(vrsfile)
# Since we want to display the position of the RGB camera, we are querying its relative location
# from the device and will apply it to the device trajectory.
T_device_RGB = provider.get_device_calibration().get_transform_device_sensor(
    "camera-rgb"
)

## Load trajectory and global points
# mps_trajectory = mps.read_closed_loop_trajectory(closed_loop_trajectory)
# points = mps.read_global_point_cloud(global_points)

## Load eyegaze
generalized_eye_gazes = mps.read_eyegaze(generalized_eye_gaze_path)

## Load hand tracking
# wrist_and_palm_poses = mps.hand_tracking.read_wrist_and_palm_poses(
#     wrist_and_palm_poses_path
# )

# Loaded data must be not empty
assert(
    # len(mps_trajectory) != 0 and
    # len(points) != 0 and
    len(generalized_eye_gazes) != 0) #and
    # len(wrist_and_palm_poses) != 0)

Loaded #EyeGazes: 7195

[MultiRecordFileReader][DEBUG]: Opened file 'eedc2456-ccee-477a-916e-70a3a15cd004_blur/eedc2456-ccee-477a-916e-70a3a15cd004_blur.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 231-1/mic activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][INFO]: streamId 281-1/gps activated
[VrsDataProvider][INFO]: streamId 282-1/wps activated
[VrsDataProvider][INFO]: streamId 283-1/bluetooth activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated
Failed to parse eye gaze vergence file: Extra column "yaw

In [2]:
import plotly.graph_objs as go
from matplotlib import pyplot as plt

# Helper function to build the frustum
def build_cam_frustum(transform_world_device):
    points = (
        np.array(
            [[0, 0, 0], [0.5, 0.5, 1], [-0.5, 0.5, 1], [-0.5, -0.5, 1], [0.5, -0.5, 1]]
        )
        * 0.6
    )
    transform_world_rgb = transform_world_device @ T_device_RGB
    points_transformed = transform_world_rgb @ points.transpose()
    return go.Mesh3d(
        x=points_transformed[0, :],
        y=points_transformed[1, :],
        z=points_transformed[2, :],
        i=[0, 0, 0, 0, 1, 1],
        j=[1, 2, 3, 4, 2, 3],
        k=[2, 3, 4, 1, 3, 4],
        showscale=False,
        visible=False,
        colorscale="jet",
        intensity=points[:, 2],
        opacity=1.0,
        hoverinfo="none",
    )

In [3]:
# # Load all world positions from the trajectory
# traj = np.empty([len(mps_trajectory), 3])
# for i in range(len(mps_trajectory)):
#     traj[i, :] = mps_trajectory[i].transform_world_device.translation()

# # Subsample trajectory for quick display
# skip = 1000
# mps_trajectory_subset = mps_trajectory[::skip]
# steps = [None]*len(mps_trajectory_subset)

# # Load each pose as a camera frustum trace
# cam_frustums = [None]*len(mps_trajectory_subset)

# for i in range(len(mps_trajectory_subset)):
#     pose = mps_trajectory_subset[i]
#     cam_frustums[i] = build_cam_frustum(pose.transform_world_device)
#     timestamp = pose.tracking_timestamp.total_seconds()
#     step = dict(method="update", args=[{"visible": [False] * len(cam_frustums) + [True] * 2}, {"title": "Trajectory and Point Cloud"},], label=timestamp,)
#     step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
#     steps[i] = step
# cam_frustums[0].visible = True
    
# # Filter the point cloud by inv depth and depth and load
# points = filter_points_from_confidence(points)
# # Retrieve point position
# point_cloud = np.stack([it.position_world for it in points])

# # Create slider to allow scrubbing and set the layout
# sliders = [dict(currentvalue={"suffix": " s", "prefix": "Time :"}, pad={"t": 5}, steps=steps,)]
# layout = go.Layout(sliders=sliders, scene=dict(bgcolor='lightgray', dragmode='orbit', aspectmode='data', xaxis_visible=False, yaxis_visible=False,zaxis_visible=False))

# # Plot trajectory and point cloud
# # We color the points by their z coordinate
# trajectory = go.Scatter3d(x=traj[:, 0], y=traj[:, 1], z=traj[:, 2], mode="markers", marker={"size": 2, "opacity": 0.8, "color": "red"}, name="Trajectory", hoverinfo='none')
# global_points = go.Scatter3d(x=point_cloud[:, 0], y=point_cloud[:, 1], z=point_cloud[:, 2], mode="markers",
#     marker={"size" : 1.5, "color": point_cloud[:, 2], "cmin": -1.5, "cmax": 2, "colorscale": "viridis",},
#     name="Global Points", hoverinfo='none')

# # draw
# plot_figure = go.Figure(data=cam_frustums + [trajectory, global_points], layout=layout)
# plot_figure.show()

In [4]:
rgb_stream_id = StreamId("214-1")
rgb_stream_label = provider.get_label_from_stream_id(rgb_stream_id)
num_rgb_frames = provider.get_num_data(rgb_stream_id)
rgb_frame = provider.get_image_data_by_index(rgb_stream_id, (int)(num_rgb_frames-5))
assert rgb_frame[0] is not None, "no rgb frame"

image = rgb_frame[0].to_numpy_array()
capture_timestamp_ns = rgb_frame[1].capture_timestamp_ns
generalized_eye_gaze = get_nearest_eye_gaze(generalized_eye_gazes, capture_timestamp_ns)
# get projection function
device_calibration = provider.get_device_calibration()
cam_calibration = device_calibration.get_camera_calib(rgb_stream_label)
assert cam_calibration is not None, "no camera calibration"

# fig, (ax1) = plt.subplots(1, 1, figsize=(12, 10))

# # Draw a cross at the projected gaze center location on the RGB image at available depth or if unavailable a 1m proxy
# depth_m = generalized_eye_gaze.depth or 1.0
# generalized_gaze_center_in_pixels = get_gaze_vector_reprojection(generalized_eye_gaze, rgb_stream_label, device_calibration, cam_calibration, depth_m)
# if generalized_gaze_center_in_pixels is not None:
#     ax1.imshow(image)
#     ax1.plot(generalized_gaze_center_in_pixels[0], generalized_gaze_center_in_pixels[1], '+', c="red", mew=1, ms=20)
#     ax1.grid(False)
#     ax1.axis(False)
#     ax1.set_title("Generalized Eye Gaze")
# else:
#     print(f"Eye gaze center projected to {generalized_gaze_center_in_pixels}, which is out of camera sensor plane.")


# #plt.show()


In [5]:
import cv2
import os
import subprocess
output_dir = "subject"

slam_stream_id = StreamId("1201-1")
slam_stream_label = provider.get_label_from_stream_id(slam_stream_id)
num_slam_frames = provider.get_num_data(slam_stream_id)
slam_frame = provider.get_image_data_by_index(slam_stream_id, (int)(1))
assert slam_frame[0] is not None, "no slam frame"

image = slam_frame[0].to_numpy_array()
prev_capture_timestamp_ns = slam_frame[1].capture_timestamp_ns
time = prev_capture_timestamp_ns / 1e9

# get projection function
device_calibration = provider.get_device_calibration()
cam_calibration = device_calibration.get_camera_calib(slam_stream_label)
assert cam_calibration is not None, "no camera calibration"

# Loop over all the frames
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, 'concat.txt'), 'w') as f:
    for i in range(1,num_slam_frames):
        slam_frame = provider.get_image_data_by_index(slam_stream_id, (int)(i))
        image = slam_frame[0].to_numpy_array()
        capture_timestamp_ns = slam_frame[1].capture_timestamp_ns
        time_diff = (capture_timestamp_ns - prev_capture_timestamp_ns) / 1e9
        time = capture_timestamp_ns / 1e9

        # Save the image
        generalized_eye_gaze = get_nearest_eye_gaze(generalized_eye_gazes, capture_timestamp_ns)
        depth_m = generalized_eye_gaze.depth or 1.0
        depth, combined_yaw, combined_pitch = (
            mps.compute_depth_and_combined_gaze_direction(
                generalized_eye_gaze.vergence.left_yaw, generalized_eye_gaze.vergence.right_yaw, generalized_eye_gaze.pitch
            )
        )
        generalized_gaze_center_in_pixels = get_gaze_vector_reprojection(generalized_eye_gaze, slam_stream_label, device_calibration, cam_calibration, depth_m)
        if len(image.shape) == 2 or image.shape[2] == 1:
            # Convert the grayscale image to a color image
            image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        else:
            # The image is already a color image
            image_color = image        
        if generalized_gaze_center_in_pixels is not None:
            # Draw a red cross on the image at the gaze center
            marker_position = (int(generalized_gaze_center_in_pixels[0]), int(generalized_gaze_center_in_pixels[1]))
            cv2.drawMarker(image_color, marker_position, color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=3)

            # Add the depth as text in the top left corner of the image
            depth_text = f"Depth: {depth:.2f} meters"
            text_position = (550, image_color.shape[0] - 10)

            for i, char in enumerate(depth_text):
                # Create an image for the character
                char_image = np.zeros_like(image_color)
                cv2.putText(char_image, char, (text_position[0], text_position[1] - i * 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Rotate the character image 90 degrees counterclockwise
                M = cv2.getRotationMatrix2D((text_position[0], text_position[1] - i * 20), 90, 1)
                char_image_rotated = cv2.warpAffine(char_image, M, (char_image.shape[1], char_image.shape[0]))

                # Add the rotated character image to the original image
                mask = char_image_rotated > 0
                image_color[mask] = char_image_rotated[mask]


        cv2.imwrite(os.path.join(output_dir, f'1201-1-{i}-{time:.3f}.jpg'), image_color)
        f.write(f'file 1201-1-{i}-{time:.3f}.jpg\n')
        # Convert the image from BGR to RGB

        # Display the image
        #plt.imshow(image_color)
        #plt.show()
        #break
        
        prev_capture_timestamp_ns = capture_timestamp_ns
# if output.mp4 exists delete it
if os.path.exists('subject.mp4'):
    os.remove('subject.mp4')
if os.path.exists('subject_rotated.mp4'):
    os.remove('subject_rotated.mp4')
# Use ffmpeg to create the video with 10 fps
subprocess.run(['ffmpeg', '-r', '10', '-f', 'concat', '-safe', '0', '-i', os.path.join(output_dir, 'concat.txt'), '-c:v', 'libx264', '-pix_fmt', 'yuv420p', 'subject.mp4'])
subprocess.run(['ffmpeg', '-r', '10', '-i', 'subject.mp4', '-vf', 'transpose=1', 'subject_rotated.mp4'])

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


CompletedProcess(args=['ffmpeg', '-r', '10', '-i', 'subject.mp4', '-vf', 'transpose=1', 'subject_rotated.mp4'], returncode=0)